In [1]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

In [21]:
# Load data
well_name = 'HW-29'
well = project.get_well(well_name)
well_data = well.data.copy()
well_data['CPORE'] = well_data['CORE_POR'] / 100
well_data['CPERM'] = well_data['CORE_PERM']
well_data.dropna(subset=['NPHI', 'RHOB', 'RT', 'GR'], inplace=True)

# Quick PP Interpretation

In [22]:
from quick_pp.lithology.carbonate import Carbonate
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix
from quick_pp.qaqc import badhole_flagging, mask_outside_threshold, neu_den_xplot_hc_correction
from quick_pp.saturation import *
from quick_pp.plotter import *
from quick_pp.permeability import *
from quick_pp.ressum import *
from quick_pp.rock_type import rock_typing, estimate_vsh_gr
from quick_pp.plotter import plotly_log
from quick_pp.utils import zone_flagging, min_max_line

## Estimate Litholoy

In [23]:
carbonate_type =  'limestone'  # 'dolostone'  #
model = 'single'  # 'double'  #
method = 'neu_den'  # 'pef_den'  #

# Clean up data
well_data = badhole_flagging(well_data)
# well_data = mask_outside_threshold(well_data, fill=True)

# Initialize lithology model
args = {
    'litho_model': 'carb',
    'dry_calc_point': (.0, 2.71),
    # 'dry_dolo_point': (.0, 2.81),
    'dry_clay_point': (.3, 2.7),
    'silt_line_angle': 116,
    'wet_clay_point': (0.43, 2.6),
    'sw_water_salinity': 5000,
    'sw_m': 1.8,
    'sw_n': 2,
    'hc_corr_angle': 50,
    'hc_buffer': 0.1,
    'ressum_cutoffs': dict(
        VSHALE=.5,
        PHIT=0,
        SWT=1
    ),
}
vsh_gr = estimate_vsh_gr(well_data['GR'], min_gr=0, max_gr=150)
carb_model = Carbonate(**args)
vclw, vcalc, vdolo = carb_model.estimate_lithology(
    well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'], vsh_gr=vsh_gr,  # pef=well_data['PEF'],
    model=model, method=method, normalize= True, carbonate_type=carbonate_type
)
args.update(carb_model.__dict__)
well.update_config(args)  # Save lithology model to well

# # Implement hydrocarbon correction
# nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
#     well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'], vsh_gr=vsh_gr,
#     dry_min1_point=args['dry_calc_point'],
#     dry_clay_point=args['dry_clay_point'],
#     corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
# )

# # Estimate lithology
# carb_model = Carbonate(**args)
# vclw, vcalc, vdolo = carb_model.estimate_lithology(
#     nphihc, rhobhc,  gr=well_data['GR'], vsh_gr=vsh_gr,  # pef=well_data['PEF'], 
#     model=model, method=method, normalize= True, carbonate_type=carbonate_type
# )

# Skip HC correction
nphihc, rhobhc = well_data['NPHI'], well_data['RHOB']

# Update ZONES
well_data = zone_flagging(well_data)
zones = well_data.ZONES

In [ ]:
neutron_density_xplot(well_data['NPHI'], well_data['RHOB'], dry_min1_point=args['dry_calc_point'], **args)

In [25]:
# neutron_density_xplot(nphihc, rhobhc, dry_min1_point=args['dry_calc_point'], **args)

In [ ]:
from quick_pp.rock_type import estimate_vsh_gr

vsh_gr1 = estimate_vsh_gr(well_data['GR'], alpha=0.1)
vsh_gr2 = estimate_vsh_gr(well_data['GR'], min_gr=0, max_gr=150)

fig, axs = plt.subplots(2, 1, figsize=(20, 3), sharex=True)

axs[0].plot(well_data.DEPTH, vsh_gr1, label='vsh_gr1')
axs[0].plot(well_data.DEPTH, vsh_gr2, label='vsh_gr2')
axs[0].set_ylim(0, 1)
axs[0].legend()

axs[1].plot(well_data.DEPTH, well_data['GR'], label='GR')
axs[1].legend()

In [27]:
# Specify depth of interest
min_depth = well_data.DEPTH.min()
max_depth = well_data.DEPTH.max()

## Estimate Porosity

In [ ]:
# Estimate porosity
phit = neu_den_xplot_poro(
    nphihc, rhobhc, model='carb',
    dry_min1_point=args['dry_calc_point'],
    dry_clay_point=args['dry_clay_point'],
)

# PHID needs unnormalized lithology
vclw_un, vcalc_un, vdolo_un = Carbonate(**args).estimate_lithology(
    nphihc, rhobhc, gr=well_data['GR'], vsh_gr=vsh_gr,  # pef=well_data['PEF'],
    model=model, method=method, normalize= False, carbonate_type=carbonate_type
)
rho_ma = rho_matrix(vclay=vclw_un, vcalc=vcalc_un, vdolo=vdolo_un)
phid = density_porosity(rhobhc, rho_ma)

plt.figure(figsize=(20, 2))
plt.plot(phit, label='PHIT')
plt.plot(phid, label='PHID')
plt.legend()

## Update Data

In [31]:
# Update data in the project
well_data['NPHI_HC'] = nphihc
well_data['RHOB_HC'] = rhobhc
well_data['VCALC'] = vcalc
well_data['VDOLO'] = vdolo
well_data['VCLW'] = vclw
well_data['PHIT'] = phit
well_data['PHID'] = phid

In [ ]:
# Plot the results
well_data.drop(columns=['VOIL', 'VGAS'], errors='ignore', inplace=True)
fig = plotly_log(well_data, depth_uom=well.depth_uom)
fig.show(config=dict(scrollZoom=True))
# fig.write_html(rf"{well_name}_log.html", config=dict(scrollZoom=True))